# Tool Pattern

* https://www.youtube.com/watch?v=0sAVI8bQdRc

<img src="https://github.com/neural-maze/agentic_patterns/blob/main/img/tool_pattern.png?raw=1" alt="Alt text" width="800"/>

---

As you may already know, the information stored in LLM weights is (usually) 𝐧𝐨𝐭 𝐞𝐧𝐨𝐮𝐠𝐡 to give accurate and insightful answers to our questions.

That's why we need to provide the LLM with ways to access the outside world. 🌍

In practice, you can build tools for whatever you want (at the end of the day they are just functions the LLM can use), from a tool that let's you access Wikipedia, another to analyse the content of YouTube videos or calculate difficult integrals using Wolfram Alpha.

The second pattern we are going to implement is the **tool pattern**.

In this notebook, you'll learn how **tools** actually work. This is the **second lesson** of the "Agentic Patterns from Scratch" series. Take a look at the first lesson if you haven't!

* [First Lesson: The Reflection Pattern](https://github.com/neural-maze/agentic_patterns/blob/main/notebooks/reflection_pattern.ipynb)

## A simple function

Take a look at this function 👇

In [ ]:
import json

def get_current_weather(location: str, unit: str):
	"""
	Get the current weather in a given location

	location (str): The city and state, e.g. Madrid, Barcelona
	unit (str): The unit. It can take two values; "celsius", "fahrenheit"
	"""
	if location == "Madrid":
		return json.dumps({"temperature": 25, "unit": unit})

	else:
		return json.dumps({"temperature": 58, "unit": unit})

Very simple, right? You provide a `location` and a `unit` and it returns the temperature.

In [ ]:
get_current_weather(location="Madrid", unit="celsius")

'{"temperature": 25, "unit": "celsius"}'

But the question is:

**How can you make this function available to an LLM?**

An LLM is a type of NLP system, so it expects text as input. But how can we transform this function into text?

## A System Prompt that works

For the LLM to be aware of this function, we need to provide some relevant information about it in the context. **I'm referring to the function name, attributes, description, etc.** Take a look at the following System Prompt.

```xml
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

Here are the available tools:

<tools> {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location location (str): The city and state, e.g. Madrid, Barcelona unit (str): The unit. It can take two values; 'celsius', 'fahrenheit'",
    "parameters": {
        "properties": {
            "location": {
                "type": "string"
            },
            "unit": {
                "type": "string"
            }
        }
    }
}
</tools>
```


As you can see, the LLM enforces the LLM to behave as a `function calling AI model` who, given a list of function signatures inside the <tools></tools> XML tags
will select which one to use. When the model decides a function to use, it will return a json like the following, representing a function call:

```xml
<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>
```


Let's see how it works in practise! 👇

In [ ]:
!pip install -q openai
!pip install -q python-dotenv
!pip install -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.0 MB/s eta 0:00:00


In [ ]:
import os
import re
import getpass
from groq import Groq
from dotenv import load_dotenv

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

MODEL = "llama3-groq-70b-8192-tool-use-preview"
GROQ_CLIENT = Groq(api_key=getpass.getpass("Enter your Groq API Key: "))

# Define the System Prompt as a constant
TOOL_SYSTEM_PROMPT = """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

Here are the available tools:

<tools> {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location location (str): The city and state, e.g. Madrid, Barcelona unit (str): The unit. It can take two values; 'celsius', 'fahrenheit'",
    "parameters": {
        "properties": {
            "location": {
                "type": "str"
            },
            "unit": {
                "type": "str"
            }
        }
    }
}
</tools>
"""

Enter your Groq API Key: ··········


Let's ask a very simple question: `"What's the current temperature in Madrid, in Celsius?"`

In [ ]:
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_SYSTEM_PROMPT
    }
]
agent_chat_history = []

user_msg = {
    "role": "user",
    "content": "What's the current temperature in Madrid, in Celsius?"
}

tool_chat_history.append(user_msg)
agent_chat_history.append(user_msg)

output = GROQ_CLIENT.chat.completions.create(
    messages=tool_chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<tool_call>
{"id": 0, "name": "get_current_weather", "arguments": {"location": "Madrid", "unit": "celsius"}}
</tool_call>


---

**That's an improvement!** We may not have the *proper* answer but, with this information, we can obtain it! How? Well, we just need to:

1. Parse the LLM output. By this I mean deleting the XML tags
2. Load the output as a proper Python dict

The function below does exactly this.

---

In [ ]:
def parse_tool_call_str(tool_call_str: str):
    pattern = r'</?tool_call>'
    clean_tags = re.sub(pattern, '', tool_call_str)

    try:
        tool_call_json = json.loads(clean_tags)
        return tool_call_json
    except json.JSONDecodeError:
        return clean_tags
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "There was some error parsing the Tool's output"

In [ ]:
parsed_output = parse_tool_call_str(output)
parsed_output

{'id': 0,
 'name': 'get_current_weather',
 'arguments': {'location': 'Madrid', 'unit': 'celsius'}}

We can simply run the function now, by passing the arguments like this 👇

In [ ]:
result = get_current_weather(**parsed_output["arguments"])

In [ ]:
result

'{"temperature": 25, "unit": "celsius"}'

**That's it!** A temperature of 25 degrees Celsius.

As you can see, we're dealing with a string, so we can simply add the parsed_output to the `chat_history` so that the LLM knows the information it has to return to the user.

In [ ]:
agent_chat_history.append({
    "role": "user",
    "content": f"Observation: {result}"
})

In [ ]:
GROQ_CLIENT.chat.completions.create(
    messages=agent_chat_history,
    model=MODEL
).choices[0].message.content

'The current temperature in Madrid is 25 degrees Celsius.'

## Implementing everything the good way

To recap, we have a way for the LLM to generate `tool_calls` that we can use later to *properly* run the functions. But, as you may imagine, there are some pieces missing:

1. We need to automatically transform any function into a description like we saw in the initial system prompt.
2. We need a way to tell the agent that this function is a tool

Let's do it!

### The `tool` decorator

We are going to use the `tool` decorator to transform any Python function into a tool. You can see the implementation [here](https://github.com/neural-maze/agentic_patterns/blob/main/src/agentic_patterns/tool_pattern/tool.py). To test it out, let's make a more complex tool than before. For example, a tool that interacts with [Hacker News](https://news.ycombinator.com/), getting the current top stories.

> Reminder: To automatically generate the function signature for the tool, we need a way to infer the arguments types. For this reason, we need to create the typing annotations.

In [ ]:
import json
import requests
from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.tool_pattern.tool_agent import ToolAgent

def fetch_top_hacker_news_stories(top_n: int):
    """
    Fetch the top stories from Hacker News.

    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API.
    Each story contains the title, URL, score, author, and time of submission. The data is fetched
    from the official Firebase Hacker News API, which returns story details in JSON format.

    Args:
        top_n (int): The number of top stories to retrieve.
    """
    top_stories_url = 'https://hacker-news.firebaseio.com/v0/topstories.json'

    try:
        response = requests.get(top_stories_url)
        response.raise_for_status()  # Check for HTTP errors

        # Get the top story IDs
        top_story_ids = response.json()[:top_n]

        top_stories = []

        # For each story ID, fetch the story details
        for story_id in top_story_ids:
            story_url = f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json'
            story_response = requests.get(story_url)
            story_response.raise_for_status()  # Check for HTTP errors
            story_data = story_response.json()

            # Append the story title and URL (or other relevant info) to the list
            top_stories.append({
                'title': story_data.get('title', 'No title'),
                'url': story_data.get('url', 'No URL available'),
            })

        return json.dumps(top_stories)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return []

ModuleNotFoundError: No module named 'agentic_patterns'

If we run this Python function, we'll obtain the top HN stories, as you can see below (the top 5 in this case).

In [ ]:
json.loads(fetch_top_hacker_news_stories(top_n=5))

[{'title': 'HPy – A better C API for Python',
  'url': 'https://hpyproject.org/'},
 {'title': 'Fuzzing 101',
  'url': 'https://github.com/antonio-morales/Fuzzing101'},
 {'title': "Arthur Whitney's one liner sudoku solver (2011)",
  'url': 'https://dfns.dyalog.com/n_sudoku.htm'},
 {'title': 'Pine martens return to Dartmoor after 150-year absence',
  'url': 'https://www.theguardian.com/uk-news/2024/oct/01/pine-martens-return-to-dartmoor-after-150-year-absence'},
 {'title': 'Gokapi: Lightweight selfhosted Firefox Send alternative with AWS S3 support',
  'url': 'https://github.com/Forceu/Gokapi'}]

To transform the `fetch_top_hacker_news_stories` function into a Tool, we can use the `tool` decorator.

In [ ]:
hn_tool = tool(fetch_top_hacker_news_stories)

The Tool has the following parameters: a `name`, a `fn_signature` and the `fn` (this is the function we are going to call, this case `fetch_top_hacker_news_stories`)

In [ ]:
hn_tool.name

'fetch_top_hacker_news_stories'

By default, the tool gets its name from the function name.

In [ ]:
json.loads(hn_tool.fn_signature)

{'name': 'fetch_top_hacker_news_stories',
 'description': '\n    Fetch the top stories from Hacker News.\n\n    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API. \n    Each story contains the title, URL, score, author, and time of submission. The data is fetched \n    from the official Firebase Hacker News API, which returns story details in JSON format.\n\n    Args:\n        top_n (int): The number of top stories to retrieve.\n    ',
 'parameters': {'properties': {'top_n': {'type': 'int'}}}}

As you can see, the function signature has been automatically generated. It contains the `name`, a `description` (taken from the docstrings) and the `parameters`, whose types come from the tying annotations. Now that we have a tool, let's run the agent.

### The `ToolAgent`

To create the agent, we just need to pass a list of tools (in this case, just one).

In [ ]:
tool_agent = ToolAgent(tools=[hn_tool])

A quick check to see that everything works fine. If we ask the agent something unrelated to Hacker News, it shouldn't use the tool.

In [ ]:
output = tool_agent.run(user_msg="Tell me your name")

In [ ]:
print(output)

I don't have a personal name. I am an AI assistant designed to provide information and assist with tasks.


Now, let's ask for specific information about Hacker News.

In [ ]:
output = tool_agent.run(user_msg="Tell me the top 5 Hacker News stories right now")


Using Tool: fetch_top_hacker_news_stories

Tool call dict: 
{'id': 0, 'name': 'fetch_top_hacker_news_stories', 'arguments': {'top_n': 5}}

Tool result: 
[{"title": "HPy \u2013 A better C API for Python", "url": "https://hpyproject.org/"}, {"title": "Fuzzing 101", "url": "https://github.com/antonio-morales/Fuzzing101"}, {"title": "Arthur Whitney's one liner sudoku solver (2011)", "url": "https://dfns.dyalog.com/n_sudoku.htm"}, {"title": "Pine martens return to Dartmoor after 150-year absence", "url": "https://www.theguardian.com/uk-news/2024/oct/01/pine-martens-return-to-dartmoor-after-150-year-absence"}, {"title": "Gokapi: Lightweight selfhosted Firefox Send alternative with AWS S3 support", "url": "https://github.com/Forceu/Gokapi"}]


In [ ]:
print(output)

Here are the top 5 Hacker News stories right now:
1. [HPy – A better C API for Python](https://hpyproject.org/)
2. [Fuzzing 101](https://github.com/antonio-morales/Fuzzing101)
3. [Arthur Whitney's one liner sudoku solver (2011)](https://dfns.dyalog.com/n_sudoku.htm)
4. [Pine martens return to Dartmoor after 150-year absence](https://www.theguardian.com/uk-news/2024/oct/01/pine-martens-return-to-dartmoor-after-150-year-absence)
5. [Gokapi: Lightweight selfhosted Firefox Send alternative with AWS S3 support](https://github.com/Forceu/Gokapi)


---
There you have it!! A fully functional Tool!! 🛠️